# Why Agents?

What is an agent?

An agent is a reference to combining models that can perform some kind of reasoning, like large language models (e.g ChatGPT, Llama2, Mistral, etc...) with tools to give it access to the real world,
so they can do things like browsing the internet, buying stuff, etc...

Ok, so, why is there so much hype around agents right now?

Because Agents are cool! Recently with the advance of LLMs, we've seen them become an amazing tool to do all sorts of things like building apps, browse the internet and more.



SOme neat examples of these kinds of agents can be found in here:

- [AutoGPT](https://github.com/Significant-Gravitas/AutoGPT)
- [GPT-Engineer](https://github.com/gpt-engineer-org/gpt-engineer)
- [BabyAGI](https://github.com/yoheinakajima/babyagi)

Now, today although they seem extremely powerful, agents are still at a very early stage in terms of readiness to deploy as products, something you can atest by listening to Andrej Karpathy talk about agents in this talk here:

- [Karpathy on Agents](https://www.youtube.com/watch?v=fqVLjtvWgq8)

This live-training is all about! Getting you excited about this amazing new technology, understanding it from the ground up but with a focus on practical applications and fun stuff you can do with them! 

# What is an Agent?

An agent is nothing more than some entity that can _think_ and _act_, that's right, in a way you're an agent! 

After all you can think and act on those thoughts like in the case of coming to this live-training:

- Thought: "I want to learn about agents"

- Action: "Go to the internet and research cool platforms where I can learn about agents"

- Thought: "O'Reilly has some awesome courses and live-trainings"

- Action: "Look up O'Reilly courses"

- Thought: "Live-trainings by instructor Lucas are awesome"

- Action: "Schedule live-training about agents with instructor Lucas Soares" (lol)

In a way this is a simplified rendition of what brought you here, obviously not necessarily in this particular order nor these particular sets of thought and action pairs. This particular way of thinking about how to structure thoughts and actions is well represented in the paper: [ReACT](https://arxiv.org/pdf/2210.03629.pdf). 

With regards to LLMs, how can bring this idea to fruition thinking about the LLM model as the reasoning and thinking engine?

We can start simple and just call the openai API to start:

In [ ]:
%pip install openai

In [ ]:
# uncomment this if running locally
#!pip install python-dotenv
# from dotenv import load_dotenv

# load_dotenv()

# Or if you are in Colab, uncoment below and add your api key
import os
os.environ["OPENAI_API_KEY"] = "your-api-key"

In [1]:
from openai import OpenAI
from IPython.display import Markdown
client = OpenAI()

def get_response(prompt_question):
    response = client.chat.completions.create(
        model="gpt-4o-mini",
        messages=[{"role": "system", "content": "You are a helpful research and programming assistant"},
                  {"role": "user", "content": prompt_question}]
    )
    
    return response.choices[0].message.content

output = get_response("Create a joke about a bald teacher explaining agents to wonderful students.")
Markdown(output)

Why did the bald teacher bring a mirror to explain agents to his wonderful students?

Because he wanted to show them how "reflection" works in decision-making!

Ok cool, so here we have three ideas of actions to perform:

- Creating directories
- Listing files
- Removing files

Let's transform them into functions that we could call just like in any type of Python-based application.

In [2]:
def create_directory(dir_name):
    os.makedirs(dir_name, exist_ok=True)

def create_file(filename):
    with open(filename, 'w'):
        pass

def list_files():
    files = os.listdir()
    for file in files:
        print(file)

Now, let's imagine that we wanted to create an agent that would perform these actions for us based on some input that we give it, how can we connect models that we know and can use today like ChatGPT, with these tools that do stuff in the real world?

To answer this question, how about we give a task to the model, and for that task we ask it to list the steps that it needs to perform to complete the task, and then for each of those steps we would ask the model to decide whether or not a function should be called to execute that task? 

In the famous paper ['Toolformer'](https://arxiv.org/pdf/2302.04761.pdf) they demonstrated that today's advanced LLMs like the gpt-series could teacha themselves how to properly call and use external tools!

Isn't that awesome???

So, let's see if we can hack our way into connecting the llm response with the functions that we want that llm to use.

In [ ]:
def get_response(prompt_question):
    response = client.chat.completions.create(
        model="gpt-4o-mini",
        messages=[{"role": "system", "content": "You are a helpful research and programming assistant"},
                {"role": "user", "content": prompt_question}]
    )
    
    return response.choices[0].message.content


Now, how can we actually put it all together so that given a task, a model can:

- Plan the task
- Execute actions to complete the task
- Know when to call a function


This is actually an interesting problem, let's understand why is that the case by trying to hack our way into putting all of these together:

In [2]:
def get_response(prompt_question, model="gpt-3.5-turbo-16k"):
    response = client.chat.completions.create(
        model=model,
        messages=[{"role": "system", "content": "You are a helpful research and programming assistant"},
                {"role": "user", "content": prompt_question}]
    )
    
    return response.choices[0].message.content

def create_directory(dir_name):
    os.makedirs(dir_name, exist_ok=True)

def create_file(filename):
    with open(filename, 'w'):
        pass

def list_files():
    files = os.listdir()
    for file in files:
        print(file)

    

task_description = "Create a folder called 'lucas-loves-pancakes'. Inside that folder, \
create a file called 'the-10-master-rules.md"

output = get_response(f"""Given this task: {task_description}, \n
                            Consider you have access to the following functions:
                            
    def create_directory(dir_name):
        os.makedirs(dir_name, exist_ok=True)

    def create_file():
        with open('test.txt', 'w'):
            pass

    def list_files():
        files = os.listdir()
        for file in files:
            print(file)
    
    Your output should be the first function to be executed to complete the task containing the necessary arguments.
    The OUTPUT SHOULD ONLY BE THE PYTHON FUNCTION CALL and NOTHING ELSE.
    """)

Markdown(output)

create_directory('lucas-loves-pancakes')

In [3]:
import os
exec(output)

In [4]:
!ls -d ./* | grep pancakes

./lucas-loves-pancakes


At this point we can start identifying a lot of issues with this approach despite our early sucess:

- Uncertainty of model's outputs can affect our ability to reliably call the functions
- We need more structured ways to prepare the inputs of the function calls
- We need better ways to put everything together (just feeding the entire functions like this makes it a very clunky and non-scalable framework for more complex cases)

There are many more issues but starting with these, we can now look at frameworks and see how they fix these issues and with that in mind understand what is behind their implementations!

I personally think this is a much better way to understand what is going on behind agents in practice rather than just use the more higher level frameworks right of the bat!

# OpenAI Functions

Ok, let's first understand how [OpenAI](https://openai.com/) the company behind ChatGPT, allows for these function call implementations in its API.

OpenAI implemented a [function calling API](https://platform.openai.com/docs/guides/function-calling) which is a standard way to connect their models to outside tools like in the very simple example we did above.

According to their [official documentation](https://platform.openai.com/docs/guides/function-calling#:~:text=The%20basic%20sequence,to%20the%20user.) the sequence of steps for function calling is as follows:
1. Call the model with the user query and a set of functions defined in the functions parameter.
2. The model can choose to call one or more functions; if so, the content will be a stringified JSON object adhering to your custom schema (note: the model may hallucinate parameters).
3. Parse the string into JSON in your code, and call your function with the provided arguments if they exist.
4. Call the model again by appending the function response as a new message, and let the model summarize the results back to the user.

Below is an example taken from their official documentation:

In [9]:
from openai import OpenAI
import json

client = OpenAI()

Let's look at how our previous model with those three simple functions: `create_directory()`, `create_file()`, and `list_files()` would be implemented using OpenAI's function calling approach:

In [10]:
import json
import subprocess
import json
import os

def create_directory(directory_name):
    """Function that creates a directory given a directory name."""
    os.mkdir(directory_name)
    return json.dumps({"directory_name": directory_name})


tool_create_directory = {
    "type": "function",
    "function": {
        "name": "create_directory",
        "description": "Create a directory given a directory name.",
        "parameters": {
            "type": "object",
            "properties": {
                "directory_name": {
                    "type": "string",
                    "description": "The name of the directory to create.",
                }
            },
            "required": ["directory_name"],
        },
    },
}

tools = [tool_create_directory]

In [11]:
import json

def run_terminal_task():
    messages = [
        {"role": "user", 
         "content": "Create a folder called 'pancakes-are-better-than-waffles'."}]
    tools = [tool_create_directory]  
    response = client.chat.completions.create(
        model="gpt-4o-mini",
        messages=messages,
        tools=tools,
        tool_choice="auto",  # auto is default, but we'll be explicit
    )
    response_message = response.choices[0].message
    tool_calls = response_message.tool_calls
    # Step 2: check if the model wanted to call a function
    
    if tool_calls:
        # Step 3: call the function
        # Note: the JSON response may not always be valid; be sure to handle errors
        available_functions = {
            "create_directory": create_directory,
        }
        messages.append(response_message)
        # Step 4: send the info for each function call and function response to the model
        for tool_call in tool_calls:
            function_name = tool_call.function.name
            function_to_call = available_functions[function_name]
            function_args = json.loads(tool_call.function.arguments)
            function_response = function_to_call(
                directory_name=function_args.get("directory_name"),
            )
            messages.append(
                {
                    "tool_call_id": tool_call.id,
                    "role": "tool",
                    "name": function_name,
                    "content": function_response,
                }
            )
        second_response = client.chat.completions.create(
            model="gpt-4o-mini",
            messages=messages,
        )
        return second_response

output = run_terminal_task()
output

ChatCompletion(id='chatcmpl-Av4WW7GV0aJr1wW2wviNFzxtNLV9W', choices=[Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content="A folder called 'pancakes-are-better-than-waffles' has been created.", refusal=None, role='assistant', audio=None, function_call=None, tool_calls=None))], created=1738165448, model='gpt-4o-mini-2024-07-18', object='chat.completion', service_tier='default', system_fingerprint='fp_72ed7ab54c', usage=CompletionUsage(completion_tokens=21, prompt_tokens=74, total_tokens=95, completion_tokens_details=CompletionTokensDetails(accepted_prediction_tokens=0, audio_tokens=0, reasoning_tokens=0, rejected_prediction_tokens=0), prompt_tokens_details=PromptTokensDetails(audio_tokens=0, cached_tokens=0)))

In [12]:
output.choices[0].message.content

"A folder called 'pancakes-are-better-than-waffles' has been created."

In [13]:
!ls -d */ | grep waffles

pancakes-are-better-than-waffles/


Great! We implemented openai function calling for creating directories! We could evolve this approach but let's stop for now.

See more info on these examples from OpenAI's [official cookbook](https://cookbook.openai.com/examples/how_to_call_functions_with_chat_models).

Now, let's implement the agent loop using a few LangChain components but implementing the loop logic ourselves!


In [14]:
from langchain_ollama import ChatOllama

llm_structured_output = ChatOllama(model="llama3.2", format="json", temperature=0)

llm_structured_output.invoke("Output names of cats!")

AIMessage(content='{ "Mittens" : "A grey and white cat with bright green eyes", \n"Muffin": "A fluffy orange tabby with a sweet disposition",\n"Bella": "A sleek black cat with a shiny coat and playful personality",\n"Pussycat": "A mischievous calico cat with a love for adventure",\n"Salem": "A mysterious grey and white cat with piercing blue eyes" }', additional_kwargs={}, response_metadata={'model': 'llama3.2', 'created_at': '2025-01-29T15:50:35.130048Z', 'done': True, 'done_reason': 'stop', 'total_duration': 2247062000, 'load_duration': 565237916, 'prompt_eval_count': 30, 'prompt_eval_duration': 821000000, 'eval_count': 86, 'eval_duration': 859000000, 'message': Message(role='assistant', content='', images=None, tool_calls=None)}, id='run-1a02d6bd-571b-4df3-b3b5-b8a7fd789d92-0', usage_metadata={'input_tokens': 30, 'output_tokens': 86, 'total_tokens': 116})

In [15]:
import os

def create_dir(folder_path):
    """
    Creates a directory given a folder path.
    """
    if not os.path.exists(folder_path):
        os.mkdir(folder_path)
    
    return f"Folder path was created at: {folder_path}"


create_dir("lucas-test-agents")

'Folder path was created at: lucas-test-agents'

In [16]:
!ls -d */ | grep test-agents

lucas-test-agents/


In [17]:
def create_file(file_path, contents=""):
    """
    Creates a file with content.
    If no content is provided it will create an empty file.
    """
    
    with open(file_path, "w") as f:
        f.write(contents)
    
    return f"A file was created at: {file_path}"

create_file("./lucas-test-agents/file-text.txt", "This is a test")

'A file was created at: ./lucas-test-agents/file-text.txt'

In [18]:
def read_file(file_path):
    """
    Reads from file given its path.
    """
    
    with open(file_path, "r") as f:
        contents = f.read()
    
    return contents

read_file("./lucas-test-agents/file-text.txt")

'This is a test'

In [19]:
tools = [create_dir, create_file, read_file]

llm_tools = llm_structured_output.bind_tools(tools)

In [20]:
from langchain_core.prompts import ChatPromptTemplate

system_prompt = f"""
You are an agent that helps users with desktop tasks like reading and writing files and creating directories. You either call a tool
from the options available: create_dir, create_file or read_file, or you return a summary of the tasks performed and at the end a string: END.
"""
prompt_template = ChatPromptTemplate.from_messages([
    ("system", system_prompt),
    ("human", "{input}")
])
llm_agent = prompt_template | llm_tools

In [21]:
llm_agent.invoke("Create file named lucas-loves-pancakes.md")

AIMessage(content='', additional_kwargs={}, response_metadata={'model': 'llama3.2', 'created_at': '2025-01-29T15:54:22.825689Z', 'done': True, 'done_reason': 'stop', 'total_duration': 898879958, 'load_duration': 30640417, 'prompt_eval_count': 329, 'prompt_eval_duration': 267000000, 'eval_count': 29, 'eval_duration': 286000000, 'message': Message(role='assistant', content='', images=None, tool_calls=None)}, id='run-26c781cc-febe-47ae-bf64-6bc9ca2b6f18-0', tool_calls=[{'name': 'create_file', 'args': {'contents': '', 'file_path': 'lucas-loves-pancakes.md'}, 'id': '4aaed786-d883-4c9a-a383-afd009551f0f', 'type': 'tool_call'}], usage_metadata={'input_tokens': 329, 'output_tokens': 29, 'total_tokens': 358})

In [22]:
output_tool_call = llm_agent.invoke("Create a file named ./test-agent.txt")
output_tool_call.tool_calls

[{'name': 'create_file',
  'args': {'contents': '', 'file_path': './test-agent.txt'},
  'id': 'b3e88340-8761-453e-8fde-10897e01d59d',
  'type': 'tool_call'}]

In [23]:
tool_mapping = {
    "create_file": create_file,
    "create_dir": create_dir,
    "read_file": read_file
}

In [24]:
def llm_call(query, observations=[], actions_taken=[]):
    """
    Calls the llm, it can return a tool call with arguments for calling different tools,
    or an output to the user.
    """
    template = """
    Imagine you are a simple assistant tasked with managing a file system. You have access to three tools:
    
    1. create_dir: Creates a new directory.
    2. create_file: Creates a new file and optionally writes content to it.
    3. read_file: Reads the contents of an existing file.

    Based on the user input and your observations, choose an action to execute. Your action must follow these guidelines:
    
    * Action Guidelines *
    1) Only one action is allowed per iteration.
    2) Be concise and specific about what to create, write, or read.
    3) Provide clear reasoning for your action.
    4) Always consider the current context before taking action.
    
    Respond in the following format:
    Thought: {{Your reasoning here}}
    Action: {{Action name with parameters}}
    
    Available Actions:
    - create_dir [directory_name]
    - create_file [file_name]; [content (optional)]
    - read_file [file_name]
    """

    # Prepare the input for the LLM
    prompt = f"""
    {template}

    User Query: {query}
    Observations: {observations}
    Actions Taken So Far: {actions_taken}
    """

    output = llm_agent.invoke(prompt)

    if output.tool_calls:
        for tool_call in output.tool_calls:
            function_name = tool_call["name"]
            function_args = tool_call.get("args", [])  # default to empty list if "args" is missing

            # Check if function_name exists in tool_mapping
            if function_name not in tool_mapping:
                print(f"Error: Unknown function name '{function_name}'")
                continue

            try:
                # Attempt to call the function with the provided arguments
                tool_output = tool_mapping[function_name](**function_args)
                actions_taken.append(function_name)
                observations.append(tool_output)
            except TypeError as e:
                # Handle errors if the number of arguments does not match the expected number for the function
                print(f"Error: Invalid arguments for function '{function_name}': {e}")
    
    
    return output, actions_taken, observations

In [25]:
llm_call("Create a folder named: lucas-tests-tool-calling")

(AIMessage(content='', additional_kwargs={}, response_metadata={'model': 'llama3.2', 'created_at': '2025-01-29T16:00:09.563409Z', 'done': True, 'done_reason': 'stop', 'total_duration': 1912807792, 'load_duration': 563661667, 'prompt_eval_count': 558, 'prompt_eval_duration': 460000000, 'eval_count': 55, 'eval_duration': 565000000, 'message': Message(role='assistant', content='', images=None, tool_calls=None)}, id='run-3d92583b-e504-4e73-8387-6f4e4eba5c77-0', tool_calls=[{'name': 'create_dir', 'args': {'folder_path': 'lucas-tests-tool-calling'}, 'id': '0a2e71c7-95b6-4a4a-b6c0-781d6117785b', 'type': 'tool_call'}], usage_metadata={'input_tokens': 558, 'output_tokens': 55, 'total_tokens': 613}),
 ['create_dir'],
 ['Folder path was created at: lucas-tests-tool-calling'])

In [26]:
llm_call("Create a file at: ./lucas-tests-tool-calling/test1.txt with the contents: Hello Lucas! You just made a tool call!")

(AIMessage(content='', additional_kwargs={}, response_metadata={'model': 'llama3.2', 'created_at': '2025-01-29T16:00:27.615399Z', 'done': True, 'done_reason': 'stop', 'total_duration': 584027917, 'load_duration': 32026375, 'prompt_eval_count': 591, 'prompt_eval_duration': 130000000, 'eval_count': 41, 'eval_duration': 420000000, 'message': Message(role='assistant', content='', images=None, tool_calls=None)}, id='run-6a1677b6-7203-4d62-bb23-cfd6e732e0f4-0', tool_calls=[{'name': 'create_file', 'args': {'contents': 'Hello Lucas! You just made a tool call!', 'file_path': './lucas-tests-tool-calling/test1.txt'}, 'id': 'c733aed2-81a3-4cd9-8ac3-97a6ec0189ba', 'type': 'tool_call'}], usage_metadata={'input_tokens': 591, 'output_tokens': 41, 'total_tokens': 632}),
 ['create_dir', 'create_file'],
 ['Folder path was created at: lucas-tests-tool-calling',
  'A file was created at: ./lucas-tests-tool-calling/test1.txt'])

In [27]:
llm_call("Read the file contents from  ./lucas-tests-tool-calling/test1.txt")

(AIMessage(content='', additional_kwargs={}, response_metadata={'model': 'llama3.2', 'created_at': '2025-01-29T16:00:35.620953Z', 'done': True, 'done_reason': 'stop', 'total_duration': 444725209, 'load_duration': 33339625, 'prompt_eval_count': 601, 'prompt_eval_duration': 142000000, 'eval_count': 26, 'eval_duration': 267000000, 'message': Message(role='assistant', content='', images=None, tool_calls=None)}, id='run-c3a0f114-2691-4cb6-80b3-cd63080a0d77-0', tool_calls=[{'name': 'read_file', 'args': {'file_path': './lucas-tests-tool-calling/test1.txt'}, 'id': 'e1c3c056-5a22-4922-bec3-4dd0cf6ddde8', 'type': 'tool_call'}], usage_metadata={'input_tokens': 601, 'output_tokens': 26, 'total_tokens': 627}),
 ['create_dir', 'create_file', 'read_file'],
 ['Folder path was created at: lucas-tests-tool-calling',
  'A file was created at: ./lucas-tests-tool-calling/test1.txt',
  'Hello Lucas! You just made a tool call!'])

In [28]:
def agent_loop(query):
    iter_count = 0
    obs = []
    acts_taken = []
    while True:
        output,obs,acts_taken = llm_call(query, obs, acts_taken)
        print(output)
        iter_count+=1
        if iter_count>=3:
            print(f"Breaking after {iter_count} iterations")
            break
        if output.content!="":
            break
    
    return output.content


agent_loop("Create a folder in current directory named 'testing-multiple-calls' and inside that folder create a file named nested-file.txt")        

content='' additional_kwargs={} response_metadata={'model': 'llama3.2', 'created_at': '2025-01-29T16:03:43.965274Z', 'done': True, 'done_reason': 'stop', 'total_duration': 581046000, 'load_duration': 34512959, 'prompt_eval_count': 572, 'prompt_eval_duration': 125000000, 'eval_count': 30, 'eval_duration': 419000000, 'message': Message(role='assistant', content='', images=None, tool_calls=None)} id='run-20cdd028-8758-4d1a-9996-356226c9061c-0' tool_calls=[{'name': 'create_dir', 'args': {'folder_path': './testing-multiple-calls'}, 'id': '2faf5536-24e3-4295-86da-139f366a1d6b', 'type': 'tool_call'}] usage_metadata={'input_tokens': 572, 'output_tokens': 30, 'total_tokens': 602}
content='' additional_kwargs={} response_metadata={'model': 'llama3.2', 'created_at': '2025-01-29T16:03:44.80329Z', 'done': True, 'done_reason': 'stop', 'total_duration': 834996584, 'load_duration': 11996959, 'prompt_eval_count': 588, 'prompt_eval_duration': 44000000, 'eval_count': 74, 'eval_duration': 777000000, 'mess

''

In [ ]:
import os

def create_dir(folder_path):
    """
    Creates a directory given a folder path.
    """
    if not os.path.exists(folder_path):
        os.mkdir(folder_path)
    
    return f"Folder path was created at: {folder_path}"


create_dir("lucas-test-agents")


'Folder path was created at: lucas-test-agents'

In [ ]:
!ls -d */

WebRover/          lucas-test-agents/ react-voice-agent/


In [ ]:
def create_file(file_path, contents=""):
    """
    Creates a file with content.
    If no content is provided it will create an empty file.
    """
    
    with open(file_path, "w") as f:
        f.write(contents)
    
    return f"A file was created at: {file_path}"

create_file("./lucas-test-agents/file-text.txt", "This is a test")

'A file was created at: ./lucas-test-agents/file-text.txt'

In [ ]:
def read_file(file_path):
    """
    Reads from file given its path.
    """
    
    with open(file_path, "r") as f:
        contents = f.read()
    
    return contents

read_file("./lucas-test-agents/file-text.txt")

'This is a test'

In [ ]:
tools = [create_dir, create_file, read_file]

llm_tools = llm_structured_output.bind_tools(tools)

In [ ]:
from langchain_core.prompts import ChatPromptTemplate
system_prompt = f"""You are an agent that helps users with desktop tasks like reading and writing files and creating directories. You either call a tool
from the options available: create_dir, create_file or read_file, or you return a summary of the tasks performed and at the end a string: END.
"""
prompt_template = ChatPromptTemplate.from_messages([
    ("system", system_prompt),
    ("human", "{input}")
])
llm_agent = prompt_template | llm_tools

In [ ]:
llm_agent.invoke("Create file named lucas-locas-pancakes.md")

AIMessage(content='', response_metadata={'model': 'llama3.2', 'created_at': '2025-01-27T21:30:19.116521Z', 'message': {'role': 'assistant', 'content': '', 'tool_calls': [{'function': {'name': 'create_file', 'arguments': {'contents': '', 'file_path': 'lucas-locas-pancakes.md'}}}]}, 'done_reason': 'stop', 'done': True, 'total_duration': 438141625, 'load_duration': 26659959, 'prompt_eval_count': 330, 'prompt_eval_duration': 130000000, 'eval_count': 30, 'eval_duration': 279000000}, id='run-2a61ea11-f765-4ea6-842e-182bc2edb27a-0', tool_calls=[{'name': 'create_file', 'args': {'contents': '', 'file_path': 'lucas-locas-pancakes.md'}, 'id': '8d96d871-6bc1-465b-9165-38233700957e', 'type': 'tool_call'}], usage_metadata={'input_tokens': 330, 'output_tokens': 30, 'total_tokens': 360})

In [ ]:
output_tool_call = llm_agent.invoke("Create a file named ./test-agent.txt")
output_tool_call.tool_calls

[{'name': 'create_file',
  'args': {'contents': '', 'file_path': './test-agent.txt'},
  'id': 'ae06a878-1490-4aaa-a766-9488c5521aed',
  'type': 'tool_call'}]

In [ ]:
tool_mapping = {
    "create_file": create_file,
    "create_dir": create_dir,
    "read_file": read_file
}

In [ ]:
def llm_call(query, observations=[], actions_taken=[]):
    """
    Calls the llm, it can return a tool call with arguments for calling different tools,
    or an output to the user.
    """
    template = """
    Imagine you are a simple assistant tasked with managing a file system. You have access to three tools:
    
    1. create_dir: Creates a new directory.
    2. create_file: Creates a new file and optionally writes content to it.
    3. read_file: Reads the contents of an existing file.

    Based on the user input and your observations, choose an action to execute. Your action must follow these guidelines:
    
    * Action Guidelines *
    1) Only one action is allowed per iteration.
    2) Be concise and specific about what to create, write, or read.
    3) Provide clear reasoning for your action.
    4) Always consider the current context before taking action.
    
    Respond in the following format:
    Thought: {{Your reasoning here}}
    Action: {{Action name with parameters}}
    
    Available Actions:
    - create_dir [directory_name]
    - create_file [file_name]; [content (optional)]
    - read_file [file_name]
    """

    # Prepare the input for the LLM
    prompt = f"""
    {template}

    User Query: {query}
    Observations: {observations}
    Actions Taken So Far: {actions_taken}
    """

    output = llm_agent.invoke(prompt)

    if output.tool_calls:
        for tool_call in output.tool_calls:
            function_name = tool_call["name"]
            function_args = tool_call.get("args", [])  # default to empty list if "args" is missing

            # Check if function_name exists in tool_mapping
            if function_name not in tool_mapping:
                print(f"Error: Unknown function name '{function_name}'")
                continue

            try:
                # Attempt to call the function with the provided arguments
                tool_output = tool_mapping[function_name](**function_args)
                actions_taken.append(function_name)
                observations.append(tool_output)
            except TypeError as e:
                # Handle errors if the number of arguments does not match the expected number for the function
                print(f"Error: Invalid arguments for function '{function_name}': {e}")
    
    
    return output, actions_taken, observations

In [ ]:
llm_call("Create a folder named: lucas-tests-tool-calling")

(AIMessage(content='', response_metadata={'model': 'llama3.2', 'created_at': '2025-01-27T22:21:11.784171Z', 'message': {'role': 'assistant', 'content': '', 'tool_calls': [{'function': {'name': 'create_dir', 'arguments': {'folder_path': 'lucas-tests-tool-calling'}}}]}, 'done_reason': 'stop', 'done': True, 'total_duration': 1901656625, 'load_duration': 568133000, 'prompt_eval_count': 558, 'prompt_eval_duration': 451000000, 'eval_count': 55, 'eval_duration': 553000000}, id='run-204906b5-3c97-4285-8b54-4c376a042e9e-0', tool_calls=[{'name': 'create_dir', 'args': {'folder_path': 'lucas-tests-tool-calling'}, 'id': 'bf8293d0-58b8-4be5-b3b6-24738374640e', 'type': 'tool_call'}], usage_metadata={'input_tokens': 558, 'output_tokens': 55, 'total_tokens': 613}),
 ['create_dir'],
 ['Folder path was created at: lucas-tests-tool-calling'])

In [ ]:
llm_call("Create a file at: ./lucas-tests-tool-calling/test1.txt with the contents: Hello Lucas! You just made a tool call!")

{'output': ('observation',
  ['A file was created at: ./lucas-tests-tool-calling/test1.txt'])}

In [ ]:
llm_call("Read the file contents from  ./lucas-tests-tool-calling/test1.txt")

{'output': ('observation', ['Hello Lucas! You just made a tool call!'])}

In [ ]:
def agent_loop(query):
    iter_count = 0
    obs = []
    acts_taken = []
    while True:
        output,obs,acts_taken = llm_call(query, obs, acts_taken)
        print(output)
        iter_count+=1
        if iter_count>=3:
            print(f"Breaking after {iter_count} iterations")
            break
        if output.content!="":
            break
    
    return output.content


agent_loop("Create a folder in current directory named 'testing-multiple-calls' and inside that folder create a file named nested-file.txt")        

content='' response_metadata={'model': 'llama3.2', 'created_at': '2025-01-27T22:21:26.57492Z', 'message': {'role': 'assistant', 'content': '', 'tool_calls': [{'function': {'name': 'create_dir', 'arguments': {'folder_path': './testing-multiple-calls'}}}]}, 'done_reason': 'stop', 'done': True, 'total_duration': 566164709, 'load_duration': 32140750, 'prompt_eval_count': 572, 'prompt_eval_duration': 136000000, 'eval_count': 30, 'eval_duration': 396000000} id='run-b03dcbe3-504f-4a74-896a-9f11ba4fadb4-0' tool_calls=[{'name': 'create_dir', 'args': {'folder_path': './testing-multiple-calls'}, 'id': '8f81c62c-0e25-4a4c-a377-e6732748866c', 'type': 'tool_call'}] usage_metadata={'input_tokens': 572, 'output_tokens': 30, 'total_tokens': 602}
content='' response_metadata={'model': 'llama3.2', 'created_at': '2025-01-27T22:21:27.36701Z', 'message': {'role': 'assistant', 'content': '', 'tool_calls': [{'function': {'name': 'create_dir', 'arguments': {'folder_path': './testing-multiple-calls'}}}]}, 'done

''

# References

- [HuggingGPT](https://github.com/microsoft/JARVIS)
- [Gen Agents](https://arxiv.org/pdf/2304.03442.pdf)
- [WebGPT](https://www.semanticscholar.org/paper/WebGPT%3A-Browser-assisted-question-answering-with-Nakano-Hilton/2f3efe44083af91cef562c1a3451eee2f8601d22)
- [LangChain](https://python.langchain.com/docs/get_started/introduction)
- [OpenAI](https://openai.com/)
- [OpenAI Function Calling](https://platform.openai.com/docs/guides/function-calling)
- [AutoGPT](https://github.com/Significant-Gravitas/AutoGPT)
- [GPT-Engineer](https://github.com/gpt-engineer-org/gpt-engineer)
- [BabyAGI](https://github.com/yoheinakajima/babyagi)
- [Karpathy on Agents](https://www.youtube.com/watch?v=fqVLjtvWgq8)
- [ReACT Paper](https://arxiv.org/abs/2210.03629)